In [ ]:
!pip install lightfm
import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset
from google.colab import files


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831161 sha256=055885bcd9437aae4f192353da607fabccceb575067747f9cb6d742df48189e6
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


In [ ]:
uploaded = files.upload()
csv_filename = list(uploaded.keys())[0]
df = pd.read_csv(csv_filename)

print("📄 Columns in your uploaded file:")
print(df.columns.tolist())

df.head()


Saving final_extended_dataset_filtered.csv to final_extended_dataset_filtered (2).csv
📄 Columns in your uploaded file:
['Category', 'Subcategory', 'Type', 'Item_Name', 'Item_Variant', 'Sales_Type', 'Source', 'Sales_Count', 'Revenue']


,Category,Subcategory,Type,Item_Name,Item_Variant,Sales_Type,Source,Sales_Count,Revenue
0,Sales,BEVERAGES,COFFEE BASED,Americano (Hot),Americano (Hot),Ala Carte,EZO QS,18,22000
1,Sales,BEVERAGES,COFFEE BASED,Americano (Hot),Americano (Hot),Ala Carte,POS,428,22000
2,Sales,BEVERAGES,COFFEE BASED,Americano (Hot),Americano (Hot),Ala Carte,GRAB,1,24000
3,Sales,BEVERAGES,COFFEE BASED,Cappuccino,Cappuccino,Ala Carte,EZO QS,14,25000
4,Sales,BEVERAGES,COFFEE BASED,Cappuccino,Cappuccino,Ala Carte,POS,162,25000


In [ ]:
uploaded = files.upload()
csv_filename = list(uploaded.keys())[0]
df = pd.read_csv(csv_filename)

print("🔎 Actual columns in your dataset:")
print(df.columns.tolist())

df.head()


Saving final_extended_dataset_filtered.csv to final_extended_dataset_filtered (3).csv
🔎 Actual columns in your dataset:
['Category', 'Subcategory', 'Type', 'Item_Name', 'Item_Variant', 'Sales_Type', 'Source', 'Sales_Count', 'Revenue']


,Category,Subcategory,Type,Item_Name,Item_Variant,Sales_Type,Source,Sales_Count,Revenue
0,Sales,BEVERAGES,COFFEE BASED,Americano (Hot),Americano (Hot),Ala Carte,EZO QS,18,22000
1,Sales,BEVERAGES,COFFEE BASED,Americano (Hot),Americano (Hot),Ala Carte,POS,428,22000
2,Sales,BEVERAGES,COFFEE BASED,Americano (Hot),Americano (Hot),Ala Carte,GRAB,1,24000
3,Sales,BEVERAGES,COFFEE BASED,Cappuccino,Cappuccino,Ala Carte,EZO QS,14,25000
4,Sales,BEVERAGES,COFFEE BASED,Cappuccino,Cappuccino,Ala Carte,POS,162,25000


In [ ]:
df = df.rename(columns={
    'Source': 'userID',
    'Item_Name': 'placeID',
    'Sales_Count': 'rating',
    'Revenue': 'price',
})


In [ ]:
df.dropna(subset=['userID', 'placeID', 'rating'], inplace=True)
df.drop_duplicates(subset=['userID', 'placeID'], inplace=True)


In [ ]:
from lightfm.data import Dataset
df['price'] = df['price'].astype(str)

df['combined_features'] = (
    df['price'] + ' ' +
    df['Category'].astype(str) + ' ' +
    df['Sales_Type'].astype(str)
)

dataset = Dataset()

dataset.fit(
    users=df['userID'].unique(),
    items=df['placeID'].unique(),
    user_features=None,
    item_features=df['combined_features'].unique()
)

(interactions, _) = dataset.build_interactions(
    df[['userID', 'placeID']].itertuples(index=False, name=None)
)

item_features = dataset.build_item_features(
    ((row['placeID'], [row['combined_features']]) for _, row in df.iterrows())
)


In [ ]:
from lightfm import LightFM

model = LightFM(loss='warp', no_components=30, item_alpha=1e-6)
model.fit(interactions, epochs=5, num_threads=2)

In [ ]:
def get_user_dict(df):
    return {user: idx for idx, user in enumerate(df['userID'].unique())}

def get_item_dict(df):
    return {item: idx for idx, item in enumerate(df['placeID'].unique())}

def recommend_items(model, interactions, user_id, user_dict, item_dict, n_items=5):
    if user_id not in user_dict:
        print(f"❌ User '{user_id}' not found in dataset.")
        return

    user_x = user_dict[user_id]
    scores = model.predict(user_ids=user_x, item_ids=np.arange(len(item_dict)))
    top_items = np.argsort(-scores)[:n_items]

    reverse_item_dict = {v: k for k, v in item_dict.items()}
    print(f"\n📋 Top {n_items} menu recommendations for user '{user_id}':")
    for i in top_items:
        print(f"- {reverse_item_dict[i]}")

In [ ]:
def analyze_menu_priority(df):
    menu_summary = df.groupby('placeID').agg({
        'rating': 'sum',
        'price': 'first',
        'Category': 'first'
    }).sort_values(by='rating', ascending=False)

    print("\n🔝 Top-selling items (Management Priority):")
    display(menu_summary.head(10))

    print("\n🔻 Low-selling items (Consider Deprioritizing):")
    display(menu_summary.tail(10))

In [ ]:
model = LightFM(loss='warp', no_components=30, item_alpha=1e-6)

for epoch in range(1, 51):  # 5 epochs total
    model.fit_partial(interactions, epochs=1, num_threads=2)
    auc = auc_score(model, interactions, num_threads=2).mean()
    print(f"Epoch {epoch}: AUC = {auc:.4f}")


Epoch 1: AUC = 0.9382
Epoch 2: AUC = 0.9465
Epoch 3: AUC = 0.9660
Epoch 4: AUC = 0.9762
Epoch 5: AUC = 0.9924
Epoch 6: AUC = 0.9994
Epoch 7: AUC = 0.9999
Epoch 8: AUC = 1.0000
Epoch 9: AUC = 1.0000
Epoch 10: AUC = 1.0000
Epoch 11: AUC = 1.0000
Epoch 12: AUC = 1.0000
Epoch 13: AUC = 1.0000
Epoch 14: AUC = 1.0000
Epoch 15: AUC = 1.0000
Epoch 16: AUC = 1.0000
Epoch 17: AUC = 1.0000
Epoch 18: AUC = 1.0000
Epoch 19: AUC = 1.0000
Epoch 20: AUC = 1.0000
Epoch 21: AUC = 1.0000
Epoch 22: AUC = 1.0000
Epoch 23: AUC = 1.0000
Epoch 24: AUC = 1.0000
Epoch 25: AUC = 1.0000
Epoch 26: AUC = 1.0000
Epoch 27: AUC = 1.0000
Epoch 28: AUC = 1.0000
Epoch 29: AUC = 1.0000
Epoch 30: AUC = 1.0000
Epoch 31: AUC = 1.0000
Epoch 32: AUC = 1.0000
Epoch 33: AUC = 1.0000
Epoch 34: AUC = 1.0000
Epoch 35: AUC = 1.0000
Epoch 36: AUC = 1.0000
Epoch 37: AUC = 1.0000
Epoch 38: AUC = 1.0000
Epoch 39: AUC = 1.0000
Epoch 40: AUC = 1.0000
Epoch 41: AUC = 1.0000
Epoch 42: AUC = 1.0000
Epoch 43: AUC = 1.0000
Epoch 44: AUC = 1.00

In [ ]:
user_dict = get_user_dict(df)
item_dict = get_item_dict(df)
recommend_items(model, interactions, 'POS', user_dict, item_dict)
analyze_menu_priority(df)


📋 Top 5 menu recommendations for user 'POS':
- LARGE Size Sirloin Local
- Es Batu
- Tempe
- Sosis
- Creamy Coco Pandan

🔝 Top-selling items (Management Priority):


,rating,price,Category
placeID,,,
Iced Brown Sugar Latte,924,27000,Sales
Mini Pizza,823,12000,Sales
Nasi Goreng Ayam Spesial,773,35000,Sales
Iced Lychee Tea,753,25000,Sales
Tahu Cabe Garam,709,24000,Sales
Choco Bun,705,9000,Sales
Iced Lemon Tea,657,19000,Sales
Organic Green Juice,549,29000,Sales
Nasi Goreng Jawa,517,32000,Sales



🔻 Low-selling items (Consider Deprioritizing):


,rating,price,Category
placeID,,,
Cheddar Cheese,1,6000,Sales
Biaya Tambahan (SURCHARGE),1,15000,Sales
New Castro,1,18000,Sales
Nogososro,1,26000,Sales
PRECISION GREEN 1 batang,1,3500,Sales
Mind,1,26000,Sales
Kopi Djammu Latte,1,23000,Sales
LARGE Size Sirloin Local,1,99000,Sales
Lidah Kucing,1,85000,Sales


In [ ]:
from lightfm.evaluation import auc_score

def evaluate_model_accuracy(model, interactions):
    """
    Evaluate model accuracy using AUC score.
    """
    print("Evaluating model accuracy...")
    auc = auc_score(model, interactions, num_threads=2).mean()
    print(f"Average AUC score: {auc:.4f}")


In [ ]:
evaluate_model_accuracy(model, interactions)


Evaluating model accuracy...
Average AUC score: 1.0000
